In [1]:
import pandas as pd
df=pd.read_csv(r'C:\Users\Tcs\Downloads\Reviews.csv')
df.head(5)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [2]:
import nltk

In [3]:
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tcs\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tcs\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:

reviews = df['Text']

reviews = reviews.dropna()


reviews = reviews[:10000]

print(reviews.head())

0    I have bought several of the Vitality canned d...
1    Product arrived labeled as Jumbo Salted Peanut...
2    This is a confection that has been around a fe...
3    If you are looking for the secret ingredient i...
4    Great taffy at a great price.  There was a wid...
Name: Text, dtype: object


In [5]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
print("Sample stopwords:", list(stop_words)[:10])

Sample stopwords: ['has', "shouldn't", 'when', 'if', 'did', 'ain', "aren't", 'from', "doesn't", 'than']


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tcs\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
import re 
from nltk.tokenize import word_tokenize



In [7]:
def preprocess_text(text):
    text=text.lower()
    text = re.sub(r'[^a-z\s]','',text)
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word not in stop_words]
    cleaned_text =' '.join(filtered_tokens)
    return cleaned_text

cleaned_reviews=reviews.apply(preprocess_text)
print(cleaned_reviews.head())

0    bought several vitality canned dog food produc...
1    product arrived labeled jumbo salted peanutsth...
2    confection around centuries light pillowy citr...
3    looking secret ingredient robitussin believe f...
4    great taffy great price wide assortment yummy ...
Name: Text, dtype: object


In [8]:
df['cleaned_text'] = df['Text'].apply(preprocess_text)
print(df[['Text', 'cleaned_text']].head())

                                                Text  \
0  I have bought several of the Vitality canned d...   
1  Product arrived labeled as Jumbo Salted Peanut...   
2  This is a confection that has been around a fe...   
3  If you are looking for the secret ingredient i...   
4  Great taffy at a great price.  There was a wid...   

                                        cleaned_text  
0  bought several vitality canned dog food produc...  
1  product arrived labeled jumbo salted peanutsth...  
2  confection around centuries light pillowy citr...  
3  looking secret ingredient robitussin believe f...  
4  great taffy great price wide assortment yummy ...  


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(cleaned_reviews)
print(f"TF-IDF matrix shape: {tfidf_matrix.shape}")


TF-IDF matrix shape: (10000, 23530)


In [10]:
from sklearn.metrics.pairwise import cosine_similarity

def retrieve_reviews(query, top_k=5):
    query_cleaned = preprocess_text(query)
    query_vec = vectorizer.transform([query_cleaned])
    similarities = cosine_similarity(query_vec, tfidf_matrix).flatten()
    top_indices = similarities.argsort()[-top_k:][::-1]
    print(f"\nTop {top_k} matching reviews for query: \"{query}\"\n")
    for i in top_indices:
        print(f"Review #{i+1} | Similarity Score: {similarities[i]:.4f}")
        print("Original:", reviews.iloc[i][:300])
        print("Cleaned :", cleaned_reviews.iloc[i][:300])
        print('-' * 100)

In [11]:
retrieve_reviews("great product with fast shipping")
retrieve_reviews("disappointed")



Top 5 matching reviews for query: "great product with fast shipping"

Review #5227 | Similarity Score: 0.5024
Original: Enjoyed the product and they also provided very fast shipping.  I'm about out and need to order more.
Cleaned : enjoyed product also provided fast shipping im need order
----------------------------------------------------------------------------------------------------
Review #8022 | Similarity Score: 0.4439
Original: The tea is good and fresh. We enjoy it. The shipping is fast and cost is reasonable.<br />I recommend this product.
Cleaned : tea good fresh enjoy shipping fast cost reasonablebr recommend product
----------------------------------------------------------------------------------------------------
Review #7074 | Similarity Score: 0.4057
Original: My daughter lives in Hawaii and sent me some great coffee from there. I have a Keurig and  was not sure how to use the new coffee. I got these fast fast and now am using my Hawaiian coffee. Great product.
Clean